# instaMT
A Jupyter notebook to play with moment tensor orientation and proportion of CLVD and ISO.

(c) 2020 Claudio Satriano - satriano@ipgp.fr

### How to run this notebook
Select "Cell->Run All" from the Jupyter menu.

In [ ]:
%matplotlib notebook
import numpy as np
from numpy import cos, sin, radians
# from obspy.imaging.beachball import beach
from obspy.imaging.mopad_wrapper import beach
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The Python code for this Jupyter notebook is by default hidden for easier reading.
To toggle on/off the code, click <a href="javascript:code_toggle()">here</a>.''')

In [ ]:
def fm_matrix(fm):
    matrix = np.array([
        [fm[0], fm[3], fm[4]],
        [fm[3], fm[1], fm[5]],
        [fm[4], fm[5], fm[2]],
    ])
    return matrix


def fm_array(fmm):
    fm = np.array([
        fmm[0, 0], fmm[1, 1], fmm[2, 2],
        fmm[0, 1], fmm[0, 2], fmm[1, 2]
    ])
    return fm


def rotate_x(angle):
    angle = radians(angle)
    matrix = np.array([
        [1, 0, 0],
        [0, cos(angle), -sin(angle)],
        [0, sin(angle), cos(angle)]
    ])
    return matrix


def rotate_z(angle):
    angle = radians(angle)
    matrix = np.array([
        [cos(angle), -sin(angle), 0],
        [sin(angle), cos(angle), 0],
        [0, 0, 1],
    ])
    return matrix

In [ ]:
def compose(clvd=0., iso=0.):
    """
    Create a reverse fault moment tensor with percentages of CLVD and ISO.
    
    Uses the North,East,Down (NED) convention
    """
    # "Cat-eye" reverse fault (strike=0, dip=45, rake=90)
    fm_dc = np.array((0, -1, 1, 0, 0, 0))
    # "Bull-eye" CLVD
    fm_clvd = np.array((-0.5, -0.5, 1., 0, 0, 0)) * np.sign(clvd)
    # ISO
    fm_iso = np.array((1., 1., 1., 0, 0, 0)) * 1./3 * np.sign(iso)
    # Let's compose the three components
    clvd = np.abs(clvd / 100.)
    iso = np.abs(iso / 100.)
    fm = (1-iso) * ((1-clvd) * fm_dc + clvd * fm_clvd) + iso * fm_iso
    # Now, rotate back to dip=0, rake=0 
    #  TODO: do why we have to use rotate_z(90) ??
    rot = np.dot(rotate_z(90), rotate_x(-45))
    fmm = fm_matrix(fm)
    fmm = np.dot(np.dot(rot, fmm), rot.T)
    return fm_array(fmm)

fm = compose()

fig, ax = plt.subplots(figsize=(3, 3))
ax.axison = False
bball = beach(fm, mopad_basis='NED')
ax.add_collection(bball)
ax.autoscale_view(tight=False, scalex=True, scaley=True)

# Moment tensor widget (NED convention)
label = widgets.Label('Moment Tensor (NED convention):')
wfm = [widgets.FloatText() for _ in range(9)]
# Make sure that tensor stays symmetric
widgets.jslink((wfm[1], 'value'), (wfm[3], 'value'))
widgets.jslink((wfm[2], 'value'), (wfm[6], 'value'))
widgets.jslink((wfm[5], 'value'), (wfm[7], 'value'))
gbox = widgets.GridBox(
    children=wfm,
    layout=widgets.Layout(
        grid_template_rows='auto auto auto',
        grid_template_columns='100px 100px 100px',
        width='300px'
    ))
mtvalues = widgets.VBox([label, gbox])

# Moment tensor widget (RTP convention)
label2 = widgets.Label('Moment Tensor (USE/RθΦ convention):')
wfm2 = [widgets.FloatText() for _ in range(9)]
# Make sure that tensor stays symmetric
widgets.jslink((wfm2[1], 'value'), (wfm2[3], 'value'))
widgets.jslink((wfm2[2], 'value'), (wfm2[6], 'value'))
widgets.jslink((wfm2[5], 'value'), (wfm2[7], 'value'))
gbox2 = widgets.GridBox(
    children=wfm2,
    layout=widgets.Layout(
        grid_template_rows='auto auto auto',
        grid_template_columns='100px 100px 100px',
        width='300px'
    ))
mtvalues2 = widgets.VBox([label2, gbox2])


def print_fmm(fmm):
    # NED convention
    wfm[0].value = np.around(fmm[0, 0], 3)  # Mxx
    wfm[1].value = np.around(fmm[0, 1], 3)  # Mxy
    wfm[2].value = np.around(fmm[0, 2], 3)  # Mxz
    wfm[4].value = np.around(fmm[1, 1], 3)  # Myy
    wfm[5].value = np.around(fmm[1, 2], 3)  # Myz
    wfm[8].value = np.around(fmm[2, 2], 3)  # Mzz
    # USE convention
    wfm2[0].value = np.around(fmm[2, 2], 3)  # Mrr
    wfm2[1].value = np.around(fmm[0, 2], 3)  # Mrt
    wfm2[2].value = np.around(-fmm[1, 2], 3) # Mrp
    wfm2[4].value = np.around(fmm[0, 0], 3)  # Mtt
    wfm2[5].value = np.around(-fmm[0, 1], 3) # Mtp
    wfm2[8].value = np.around(fmm[1, 1], 3)  # Mpp


@widgets.interact(
    strike=(0., 360., 1.), dip=(-90., 90., 1.), rake=(-180., 180., 1.),
    clvd=(-100., 100., 1.), iso=(-100., 100., 1.))
def rotate(strike=0., dip=0., rake=0., clvd=0., iso=0.):
    fm = compose(clvd, iso)
    rot = np.dot(rotate_z(strike), rotate_x(dip))
    rot = np.dot(rot, rotate_z(-rake))
    fmm = fm_matrix(fm)
    fmm = np.dot(np.dot(rot, fmm), rot.T)
    print_fmm(fmm)
    for _c in ax.collections:
        _c.remove()
    try:
        bball = beach(fm_array(fmm), mopad_basis='NED')
        ax.add_collection(bball)
        fig.canvas.draw_idle()
    except:
        pass

    
def from_mt(m11=0, m22=0, m33=0, m12=0, m13=-1, m23=0):
    fm = (m11, m22, m33, m12, m13, m23)
    for _c in ax.collections:
        _c.remove()
    try:
        bball = beach(fm, mopad_basis='NED')
        ax.add_collection(bball)
        fig.canvas.draw_idle()
    except:
        pass
    
display(mtvalues)
display(mtvalues2)